In [ ]:
import os

import numpy as np
import cv2

from matplotlib import pyplot as plt

In [ ]:
folder = "dataset/"

haarcascade_frontalface_default_path = "haarcascade_frontalface_default.xml"
haarcascade_eye_path = "haarcascade_eye.xml"

In [ ]:
face_cascade = cv2.CascadeClassifier(haarcascade_frontalface_default_path)
eye_cascade = cv2.CascadeClassifier(haarcascade_eye_path)

# Créer un nouveau dataset

Plan :
- Créer le répertoire si la personne n'existe pas dans les datasets. Sinon, compter le nombre de fichiers pour ne pas écraser de photos déjà présentes.
- Récupérer la vidéo de la caméra.
- Sauvegarder une image sur 5 pour éviter d'avoir des images trop similaires.

In [ ]:
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

person_name = "test"

path = folder + "/" + person_name

if not os.path.exists(path):
    os.makedirs(path)
    image_label = 0
else:
    image_label = len(
        [
            f for f in os.listdir(path)
            if f.endswith('.jpg')
            and os.path.isfile(os.path.join(path, f))
        ]
    ) + 1
    
ret = True
current_iteration = 0

while ret == True:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect face in the picture in grayscale
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for x, y, w, h in faces:
        roi_color = img[y:y+h, x:x+w]
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Save 1 frame for 5 taken. This will constitute the dataset
        if current_iteration%5 == 0:
            export_image = cv2.resize(roi_color, (250, 250))
            filename = "{}/{}/{}_{}.jpg".format(
                    folder, person_name, person_name, image_label
                )
            cv2.imwrite(filename, export_image)
            image_label += 1
        current_iteration += 1
    
    cv2.imshow('Save new dataset', img)    
    if cv2.waitKey(1) == 27:
        ret = False
        break

cap.release()
cv2.destroyAllWindows()

# Entraînement

Plan :
- Charger le dataset
- Entraîner le modèle d'OpenCv : face.LBPHFaceRecognizer
- Récupérer la vidéo de la webcam
- Faire coïncider les noms des personnes et des visages.

In [ ]:
#Loading datasets
faces = []
labels = []
labels_string = []
current_label = 0

# Load every image in the dataset folder. For each picture, theirs parents directory
# will be the label of the image.
for element in os.listdir(folder):
    path = folder + element + "/"
    if os.path.isdir(path):
        for picture in os.listdir(path):
            labels.append(current_label)
            faces.append(
                    cv2.imread(path + picture, cv2.IMREAD_GRAYSCALE)
                )
    labels_string.append(element)
    current_label += 1

### Entraînement du modèle d'OpenCv

In [ ]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(faces, np.array(labels))

### Récupérer la vidéo et mettre des noms sur des visages

In [ ]:
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

ret = True

while ret == True:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Face detection
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for x, y, w, h in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y+4:y+h-4, x+4:x+w-4]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        for ex, ey, ew, eh in eyes:
            cv2.rectangle(
                    roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2
                )
        
        # Prediction
        label = face_recognizer.predict(roi_gray)
        # The label is currently a number. Here's the match between numbers and string names
        label = labels_string[label[0]]
        # Put the label on the picture. Origin is the top left corner of the face rectangle.
        img = cv2.putText(img, label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0))
    
    cv2.imshow('Affichage Resultats', img)

    c = cv2.waitKey(1)
    
    if c == 27:
        ret = False
        break

cap.release()
cv2.destroyAllWindows()